# ResNet [He et al., 2015]

**ILSVRC 2015 winner (3.6% top 5 error)**

## Layer 깊이 비교
- AlexNet, 8 layers
- VGG, 19 layers
- ResNet, **153 layers**

**Q1) layer가 기존 model보다 훨씬 깊다. 과연 학습이 잘 진행될 수 있을까?**

**A1) 아래 그림 참고**

<img align="center" width="400" height="400" src="https://github.com/DeepHaeJoong/torch_stury/blob/master/CNN/image/Lab-10-6-1.jpg?raw=true">

<img align="center" width="400" height="400" src="https://github.com/DeepHaeJoong/torch_stury/blob/master/CNN/image/Lab-10-6-2.jpg?raw=true">

##  A practical design of going deeper

- 문제점을 해결하기위해 아래와 같은 **"BasicBlock"**과 **"bottleneck"**이라는 구조사용

<img align="center" width="500" height="500" src="https://github.com/DeepHaeJoong/torch_stury/blob/master/CNN/image/Lab-10-6-3.jpg?raw=true">

## 실험 결과

- **`(a) orginal`** 구조가 가장 좋은 성능을 내었으며, 여기서는 이것을 구현할 예정

<img align="center" width="700" height="700" src="https://github.com/DeepHaeJoong/torch_stury/blob/master/CNN/image/Lab-10-6-4.jpg?raw=true">

# torchvision.models.resnet

- resnet(18, 34, 50, 101, 152)를 만들 수 있도록 되어있음
- 3 x 224 x 224 입력을 기준으로 만들도록 되어 있음
- input size가 다른 경우 ResNet를 적용하려면 어떻게 해야할까?

# 직접 구현해보기

In [37]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [8]:
def conv3x3(in_planes, out_planes, stride = 1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, bias=False)

def conv1x1(in_planes, out_planes, stride = 1):
    """1x1 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

<img align="center" width="700" height="700" src="https://github.com/DeepHaeJoong/torch_stury/blob/master/CNN/image/Lab-10-6-5.jpg?raw=true">

In [9]:
class BasicBlock(nn.Module):
    expansion = 1
    
    def __init__(self, inplanes, planes, stride=1, downsampe=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace = True)
        self.conv2 = conv3x3(inplanes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
        
    
    def forward(self, x):
        
        identity = x
        
        out = self.conv1(x) # 3x3 , stride = stride
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out) # 3x3 , stride = 1
        out = self.bn2(out)
        
        """
        # stride = 2 인 경우
        input x        => x.shape = 3 x 64 x 64
        identity.shape => 3 x 64 x 64
        out.shape      => 3 x 32 x 32
        out.shape      => 3 x 32 x 32
        
        아래 코드가 없으면 'out += identity'에서 x(3 x 32 x 32) 와 identity(3 x 64 x 64) 계산 불가능!
        """
        if self.downsample is not None: 
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out

In [31]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)                #conv1x1(64,64)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)          #conv3x3(64,64)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion) #conv1x1(64,256)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x) # 1x1 stride = 1
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out) # 3x3 stride = stride 
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out) # 1x1 stride = 1
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

<img align="center" width="700" height="700" src="https://github.com/DeepHaeJoong/torch_stury/blob/master/CNN/image/Lab-10-6-6.jpg?raw=true">

In [52]:
(224-7 +(2*3))/2 + 1

112.5

In [44]:
class ResNet(nn.Module):
    # model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    def __init__(self, block, layers, num_class=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        
        self.inplanes = 64  # _make_layer 에서 사용!

        '''
        layer_name = conv1
        input = [3 x 224 x 224]
        '''
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # ouputs = self.conv1(inputs)
        # ouputs.shape = [64 x 112 x 112]
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        

        '''
        layer_name = conv2_x, conv3_x, conv4_x, conv5_x
        input = [64 x 112 x 112]
        ''' 
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # ouputs = self.maxpool(inputs)
        # ouputs.shape = [64 x 56 x 56]
        
        self.layer1 = self._make_layer(block, 64, layers[0])             # layers[0] = 3
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)  # layers[1] = 4
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)  # layers[2] = 6
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)  # layers[3] = 3
        
        
        '''
        avg -> fc -> softmax
        '''
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_class)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        
        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
                    
    #--------------------------------------------------------------------                    
    # self.inplanes = 64                    
    # self._make_layer(Bottleneck, 64, layers[0])  # layers[0] = 3
    # self._make_layer(block=Bottleneck, planes=64, blocks=3, stride=1)
    #--------------------------------------------------------------------
    def _make_layer(self, block, planes, blocks, stride=1):
        
        downsample = None
        
        # channel을 맞추기 위해서도 사용!
        if stride != 1 or self.inplanes != planes * block.expansion:     # stride != 1 or 64 != 64 * 4 (256) -> True 실행
            downsample = nn.Sequential( 
                conv1x1(self.inplanes, planes * block.expansion, stride), # conv1x1(64, 256(64 * 4) , 1), 
                nn.BatchNorm2d(planes * block.expansion)                  # BatchNorm2d(256(64 * 4))
            )
        
        
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        # layers.append(Bottleneck(64, 64, 1, downsample))
        
        self.inplanes = planes * block.expansion                         # self.inplanes 가 64 -> 64 * 4 = 256로 변경
        
        for _ in range(1, blocks):
        # for _ in range(1, 3):
            layers.append(block(self.inplanes, planes))                  # 2번 반복!
        
        
        return nn.Sequential(*layers)
        
    '''
    self.layer1 = [
        Bottleneck(64, 64, 1, downsample)
       ,Bottleneck(256, 64)
       ,Bottleneck(256, 64)
    ]
    self.layer2 = [
        Bottleneck(256, 128, 2, downsample)
       ,Bottleneck(512, 128)
       ,Bottleneck(512, 128)
       ,Bottleneck(512, 128)       
    ]
    self.layer3 = [
        Bottleneck(512, 256, 2, downsample)
       ,Bottleneck(1024, 256)
       ,Bottleneck(1024, 256)
       ,Bottleneck(1024, 256)      
       ,Bottleneck(1024, 256)  
       ,Bottleneck(1024, 256)  
    ]
    self.layer4 = [
        Bottleneck(1024, 512, 2, downsample)
       ,Bottleneck(2048, 128)
       ,Bottleneck(2048, 128)
    ]
    '''
    
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x

In [45]:
def resnet50(pretrained=False, **kwargs):
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)  # 1 (conv1) + (3 + 4 + 6 + 3)*3 + 1(fc) = 1 + 48 + 1 = 50
    return model

In [46]:
res = resnet50()

In [47]:
res

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [51]:
def resnet18(pretrained=False, **kwargs):
    model = ResNet(BasicBlock, [2, 2, 2, 2],**kwargs) # 1 (conv1) + (2 + 2 + 2 + 2) * 2 + 1(fc) = 1 + 16 + 1 = 18
    return model